In [301]:
import numpy as np
import cnfgen
from pysat.formula import CNF
from pysat.solvers import Glucose3
import pickle
import random
import glob
from os.path import join

In [302]:
n = 10 # np.random.randint(0,1000)
print("n",n)
p = np.random.rand(n) # this is the probability to sample a 1
print("p", p)
x = np.random.rand(m)
print("x", x)
m = 50 # np.random.randint(0,2000)
print("m", m)

# matrix = np.random.randint(-1,2,size=n*m).reshape((n,m))
# print(matrix)


n 10
p [0.55897513 0.01597514 0.68792217 0.01150217 0.19959272 0.58963945
 0.52355941 0.61619128 0.86183825 0.94287605]
x [0.60402926 0.39090879 0.32095968 0.004804   0.81401961 0.22385519
 0.32871447 0.09752145 0.71752304 0.68982303 0.90985924 0.40172597
 0.44131053 0.89985748 0.45792392 0.42490162 0.50648611 0.47601032
 0.52932899 0.11759561 0.03772866 0.25669448 0.0432089  0.10687823
 0.86987441 0.50293194 0.99828415 0.79315153 0.09873366 0.23768438
 0.48792278 0.12573734 0.92265333 0.68152417 0.33143951 0.33162638
 0.97180389 0.33277835 0.47609381 0.22748317 0.94608513 0.60456242
 0.36408505 0.48940411 0.07388775 0.0580182  0.53899008 0.89185135
 0.11697559 0.28199723]
m 50


In [303]:
def find_prob_matrix(matrix, p):
    p_double = np.array([p,1-p]).transpose()
    positive = abs(matrix * np.array([p_double[:,0]]).transpose() - abs(matrix) * np.array([p_double[:,0]]).transpose())/2
    negative = abs(matrix * np.array([p_double[:,1]]).transpose() + abs(matrix) * np.array([p_double[:,1]]).transpose())/2
    mask = np.where(np.sum(abs(matrix), axis = 0)!=0, 1, 0)*np.ones(matrix.shape)
    zeros = (np.ones(np.shape(matrix)) - abs(matrix)) * mask
    return negative + positive + zeros

def find_induced_probs(matrix, p):
    return np.prod(find_prob_matrix(matrix, p), axis = 0)

def find_neighbors(matrix, conflicting_only = False):
    occurencies_matrix =abs(matrix)
    neighbors_indices_list = []
    neighbors_number_list = []
    for i in range(np.shape(occurencies_matrix)[1]):
        target = matrix[:,i][:, None]

        if conflicting_only == False:
            a = np.where(np.sum(occurencies_matrix[:,0:i] * abs(target), axis = 0)!=0, 1, 0)
            b = np.where(np.sum(occurencies_matrix[:,i+1:np.shape(occurencies_matrix)[1]] * abs(target), axis = 0)!=0, 1, 0)
        if conflicting_only == True:
            a = np.where(np.sum(((occurencies_matrix[:,0:i] * abs(target)) - (matrix[:, 0:i] * target))/2, axis = 0) != 0 , 1, 0)
            b = np.where(np.sum(((occurencies_matrix[:,i+1:np.shape(occurencies_matrix)[1]] * abs(target)) - (matrix[:, i+1:np.shape(occurencies_matrix)[1]] * target))/2, axis = 0) != 0, 1, 0)
        
        a = np.hstack((a,[0]))      
        remainder = np.hstack((a, b))
        neighbors_number = np.sum(remainder)
        neighbors_indices = np.argwhere(remainder != 0).transpose()[0]
        neighbors_number_list.append(neighbors_number)
        neighbors_indices_list.append(neighbors_indices)

    return(neighbors_indices_list,neighbors_number_list)

def get_rhs(matrix, x, conflicting_only = False):
    neighbors_indices_list,neighbors_number_list = find_neighbors(matrix, conflicting_only=conflicting_only)
    m = len(neighbors_indices_list)
    rhs_array = np.zeros(m)
    for current_clause in range(m):
        rhs = x[current_clause]
        for neighbor in neighbors_indices_list[current_clause]:
            rhs = rhs * (1-x[neighbor])
        rhs_array[current_clause] = rhs
    return rhs_array

In [304]:
def check_LLL_condition(matrix, p, x, conflicting_only = False):
    induced_probs = find_induced_probs(matrix, p)
    rhs = get_rhs(matrix, x, conflicting_only=conflicting_only)
    # print(induced_probs - rhs)
    # print(np.less_equal(induced_probs - rhs, 0))
    # print(np.all(np.less_equal(induced_probs - rhs, 0)))
    return np.all(np.less_equal(induced_probs - rhs, 0))

def check_uniqueness_condition(matrix):
    unique_matrix = np.unique(matrix, axis = 1)
    return ((matrix.shape[1] - np.sum(np.where(np.sum(matrix, axis= 0)!= 0, 0, 1))) - (unique_matrix.shape[1] - np.sum(np.where(np.sum(unique_matrix, axis= 0)!= 0, 0, 1))) == 0)

def get_LLL_matrix(p, x, conflicting_only = False, TIMEOUT = 1000):
    n = len(p)
    m = len(x)
    matrix = np.zeros((n,m))
    t = 0
    while t < TIMEOUT:
        available_indices = np.argwhere(matrix == 0)
        if len(available_indices) != 0:
            chosen_index = random.choice(available_indices)
            prob_pick = p[chosen_index[0]]
            old_matrix = np.copy(matrix)
            choice = np.random.choice([int(-1),int(1)], p = [1- prob_pick, prob_pick]) # p is the prob to sample 1
            matrix[chosen_index[0],chosen_index[1]] = choice

            #induced_probs = find_induced_probs(matrix, p)
            #rhs = get_rhs(matrix, x, conflicting_only=conflicting_only)
            check_LLL = check_LLL_condition(matrix, p, x, conflicting_only=conflicting_only)
            check_uniqueness = check_uniqueness_condition(matrix)
            #if not np.all(np.greater_equal(0, induced_probs - rhs)):
            if not np.all([check_LLL, check_uniqueness]):
                #print("matrix", np.allclose(old_matrix, matrix))
                matrix = old_matrix
                t += 1
                # print("t =", t)
            
        else:
            t = TIMEOUT
        # print(check_LLL_condition(matrix, p, x, conflicting_only=conflicting_only))
    assert check_LLL_condition(matrix, p, x, conflicting_only=conflicting_only) == True
    return np.unique(matrix, axis = 1)

matrix = get_LLL_matrix(p, x, conflicting_only= False, TIMEOUT= 1000)
print(matrix)
print(np.shape(matrix))

[[-1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1. -1. -1. -1. -1. -1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  1. -1.  0.  0.  0.  0.  0.]
 [-1.  0. -1. -1. -1.  0. -1.  0.  0. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1. -1. -1.  0.  0.  0.  0.  0.]
 [-1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.]
 [ 1.  0. -1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  0.]]
(10, 14)


In [ ]:
# idea from Paul:

def satisfies_lll(matrix, p):
    d = find_max_number_of_neighbors(matrix)
    induced_constraint_violation_probs = find_induced_probs(matrix, p)
    max_p = np.max(induced...)
    return d*max_p*np.e <=1


def cleverly_readjust_p_and_matrix(matrix,p):
    max_d, constraint_node_idx = find_max_degree_from_constraint_nodes(matrix)
    edges = edges[receiver=constraint_node_idx]
    to_drop = np.random.choose(edges,1)
    matrix = remove_edges(to_drop, matrix)
    return matrix,p

while not satisfies_lll(matrix, p):
    p, matrix = cleverly_readjust_p_and_matrix(matrix, p)

return matrix, p